In [ ]:
from google.colab import files
uploaded = files.upload()

Saving out.csv to out (1).csv
Saving test.csv to test (1).csv


In [ ]:
!pip3 install flair
import re
#import emoji
import nltk
import en_core_web_sm
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

nltk.download('vader_lexicon')
nltk.download('words')
words = set(nltk.corpus.words.words())
nlp = en_core_web_sm.load()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)
#> '3.6.0'

# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

3.6.0
[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['out.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [ ]:
def cat(x):
    if  x < 85 :
        return "1"
    if x < 90 and x >= 85:
        return "2"
    if x < 95 and x>= 90:
        return "3"
    if x>=95:
        return "4"


df2['category'] = df2['points'].apply(lambda x: cat(x))

In [ ]:
df2.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,category
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,4
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,4
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,4
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,4
4,18,France,Coming from a seven-acre vineyard named after ...,Le Pigeonnier,95,290.0,Southwest France,Cahors,NaN,Malbec,Château Lagrézette,4


In [ ]:
def cleaner(comment):
    comment = re.sub("@[A-Za-z0-9]+","",comment) #Remove @ sign
    comment = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", comment) #Remove http links
    comment = re.sub(r'[^\w\s]','',comment)
    comment = comment.lower()
    comment = re.sub(r'\d+', '',comment) #remove numbers
    comment = " ".join(comment.split())
    #comment = ''.join(c for c in comment if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    comment = comment.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    comment = " ".join(w for w in nltk.wordpunct_tokenize(comment) 
         if w.lower() in words or not w.isalpha())
    return comment

In [ ]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [ ]:
df2['processed_description'] = df2['description'].map(lambda x: cleaner(x))
df2['lemmatised_content'] = df2['processed_description'].apply(lambda x: lemmatizer(x))
df2['lemmatised_content']=df2['lemmatised_content'].str.replace('-PRON-', '')
df2['stemming_content'] = df2['lemmatised_content']

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df2['final_content'] = df2['stemming_content'].apply(stem_sentences)

In [ ]:

import flair
from flair.models import TextClassifier
from segtok.segmenter import split_single
from flair.data import Sentence

def make_sentences(text):
    """ Break apart text into a list of sentences """
    sentences = [sent for sent in split_single(text)]
    return sentences

def predict(sentence):
    """ Predict the sentiment of a sentence """
    if sentence == "":
        return 0
    text = Sentence(sentence)
    # stacked_embeddings.embed(text)
    classifier.predict(text)
    value = text.labels[0].to_dict()['value'] 
    if value == 'POSITIVE':
        result = text.to_dict()['labels'][0]['confidence']
    else:
        result = -(text.to_dict()['labels'][0]['confidence'])
    return round(result, 3)

def get_scores(sentences):
    """ Call predict on every sentence of a text """
    results = []
    for i in range(0, len(sentences)): 
        results.append(predict(sentences[i]))
    return results

     |████████████████████████████████| 61kB 4.6MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
#flair
classifier = TextClassifier.load('en-sentiment')

df2['sentences'] = df2.final_content.apply(make_sentences)
df2['flair_scores'] = df2.sentences.apply(get_scores)
df2 = df2.drop(columns='sentences')
df2['flair_scores'] = [','.join(map(str, l)) for l in df2['flair_scores']]
df2["flair_scores"] = pd.to_numeric(df2["flair_scores"], downcast="float")
conditions = [
(df2['flair_scores'] >= 0.05),
(df2['flair_scores'] <= -0.05),
((df2['flair_scores'] < 0.05) & (df2['flair_scores'] > -0.05))
]

values = ['positive', 'negative', 'neutral']

df2['flair_sentiment'] = np.select(conditions, values)

2021-03-31 02:04:19,824 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [ ]:
#vader

sentiment_analyser = SentimentIntensityAnalyzer()

sentiments_vader = []

for i in df2.final_content:
    sentiments_vader.append(sentiment_analyser.polarity_scores(i).get('compound'))
    
sentiments_vader = np.asarray(sentiments_vader)

df2['sentiment_vader'] = pd.Series(data=sentiments_vader)
df2 = df2.reset_index(drop=True)

conditions = [
    (df2['sentiment_vader'] >= 0.05),
     (df2['sentiment_vader'] <= -0.05),
    ((df2['sentiment_vader'] < 0.05) & (df2['sentiment_vader'] > -0.05))
    ]

values = ['positive', 'negative', 'neutral']

df2['vader'] = np.select(conditions, values)


In [ ]:
#textblob

sentiments_textblob = []

for review in df2.final_content:
    review = TextBlob(review)
    sentiments_textblob.append(review.sentiment.polarity)

sentiments_textblob = np.asarray(sentiments_textblob)   

df2['sentiment_textblob'] = pd.Series(data=sentiments_textblob)
df2= df2.reset_index(drop=True)
conditions = [
    (df2['sentiment_textblob'] > 0),
    (df2['sentiment_textblob'] < 0),
    (df2['sentiment_textblob'] == 0)
    ]

values = ['positive', 'negative', 'neutral']

df2['textblob'] = np.select(conditions, values)


In [ ]:
#final sentiment
df2['final_sentiment']=''
for data in range(0,len(df2)):
    lst1 = []
    flair = df2['flair_sentiment'][data]
    vader = df2['vader'][data]
    textblob = df2['textblob'][data]
    lst1.append(flair)
    if vader in lst1:
        df2['final_sentiment'][data] = vader
    else:
        lst1.append(vader)
        if textblob in lst1:
            df2['final_sentiment'][data] = textblob
        else:
            df2['final_sentiment'][data] = ('unknown')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df2.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,category,processed_description,lemmatised_content,stemming_content,final_content,flair_scores,flair_sentiment,sentiment_vader,vader,sentiment_textblob,textblob,final_sentiment
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,4,this tremendous varietal wine from and was age...,this tremendous varietal wine from and be age ...,this tremendous varietal wine from and be age ...,thi tremend variet wine from and be age over t...,1.000,positive,0.8750,positive,0.272222,positive,positive
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,4,ripe of fig blackberry and cassis are and by a...,ripe of fig blackberry and cassis be and by a ...,ripe of fig blackberry and cassis be and by a ...,ripe of fig blackberri and cassi be and by a o...,0.999,positive,0.7845,positive,0.364583,positive,positive
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,4,mac the memory of a wine once made by his moth...,mac the memory of a wine once make by mother ...,mac the memory of a wine once make by mother ...,mac the memori of a wine onc make by mother in...,0.943,positive,0.0000,neutral,-0.150000,negative,unknown
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,4,this spent in new oak and fruit from aurora an...,this spend in new oak and fruit from aurora an...,this spend in new oak and fruit from aurora an...,thi spend in new oak and fruit from aurora and...,1.000,positive,0.2023,positive,0.059091,positive,positive
4,18,France,Coming from a seven-acre vineyard named after ...,Le Pigeonnier,95,290.0,Southwest France,Cahors,NaN,Malbec,Château Lagrézette,4,coming from a vineyard after the on the proper...,come from a vineyard after the on the property...,come from a vineyard after the on the property...,come from a vineyard after the on the properti...,0.999,positive,0.9001,positive,0.361932,positive,positive


In [ ]:
cat1_pos=df['Words'][0]
cat1_neg=df['Words'][1]
cat2_pos=df['Words'][2]
cat2_neg=df['Words'][3]
cat3_pos=df['Words'][4]
cat3_neg=df['Words'][5]
cat4_pos=df['Words'][6]
cat4_neg=df['Words'][7]

In [ ]:
documents = [cat1_pos, cat1_neg, cat2_pos, cat2_neg, cat3_pos, cat3_neg, cat4_pos, cat4_neg]


In [ ]:
df2['soft_consine_score']=''
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
cat1_pos_1 = dictionary.doc2bow(simple_preprocess(cat1_pos))
cat1_neg_1  = dictionary.doc2bow(simple_preprocess(cat1_neg))
cat2_pos_1  = dictionary.doc2bow(simple_preprocess(cat2_pos))
cat2_neg_1  = dictionary.doc2bow(simple_preprocess(cat2_neg))
cat3_pos_1  = dictionary.doc2bow(simple_preprocess(cat3_pos))
cat3_neg_1  = dictionary.doc2bow(simple_preprocess(cat3_neg))
cat4_pos_1  = dictionary.doc2bow(simple_preprocess(cat4_pos))
cat4_neg_1  = dictionary.doc2bow(simple_preprocess(cat4_neg))
for i in range(0,len(df2['final_content'])):
  df2['soft_consine_score'][i]=dictionary.doc2bow(simple_preprocess(df2['final_content'][i]))

sentences = [cat1_pos_1, cat1_neg_1, cat2_pos_1, cat2_neg_1, cat3_pos_1, cat3_neg_1, cat4_pos_1, cat4_neg_1]
sentences_name = ['cat1_pos_1', 'cat1_neg_1', 'cat2_pos_1', 'cat2_neg_1', 'cat3_pos_1', 'cat3_neg_1', 'cat4_pos_1', 'cat4_neg_1']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df2['soft_consine_score']=''
output=[]
max_list=[]
for x in range(0,len(df2['final_content'])):
  for i in range(0,len(sentences)):
    output.append(softcossim(df2['soft_consine_score'][x], sentences[i], similarity_matrix))
  a=output.index(max(output))
  max_list.append(str(output[a])+','+sentences_name[a])
  df2['soft_consine_score'][x]=sentences_name[a]
  output=[]
  
print(max_list)
#a=output.index(max(output))
#print (str(output[a])+","+ sentences_name[a])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dis

['0.0,cat1_pos_1', '0.4753323718949444,cat1_pos_1', '0.6720478824795785,cat4_pos_1', '0.6188490842527788,cat4_pos_1', '0.766166879005781,cat4_pos_1', '0.0,cat1_pos_1', '0.6045654245515709,cat1_pos_1', '0.6572753297374403,cat4_pos_1', '0.6884362032505015,cat1_neg_1', '0.6425427551518167,cat4_pos_1', '0.7061194987488802,cat3_pos_1', '0.49545120478738025,cat3_pos_1', '0.6719181510458573,cat4_neg_1', '0.5765949056985467,cat4_neg_1', '0.5446684397926499,cat2_neg_1', '0.6462676014711098,cat2_pos_1', '0.36857252283308994,cat1_pos_1', '0.5684736979982327,cat4_pos_1', '0.5581831732658743,cat4_neg_1', '0.0,cat1_pos_1', '0.6215090616269572,cat4_neg_1', '0.4813856942946322,cat2_pos_1', '0.3914336248817945,cat2_pos_1', '0.6054172363196206,cat3_pos_1', '0.4813856942946322,cat2_pos_1', '0.6087462863616213,cat3_pos_1', '0.5730396278798813,cat1_pos_1', '0.4862313354616186,cat3_pos_1', '0.5857492304773676,cat2_pos_1', '0.6992170601681368,cat3_pos_1', '0.773148173652464,cat3_neg_1', '0.6248840683912951,c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dis

In [ ]:
df2['predict_category']=''
for i in range(0,len(df2['soft_consine_score'])):
  if 'cat1' in df2['soft_consine_score'][i]:
    df2['predict_category'][i]='1'
  if 'cat2' in df2['soft_consine_score'][i]:
    df2['predict_category'][i]='2'
  if 'cat3' in df2['soft_consine_score'][i]:
    df2['predict_category'][i]='3'
  if 'cat4' in df2['soft_consine_score'][i]:
    df2['predict_category'][i]='4'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/py

In [ ]:
df2['predict_sentiment']=''
for i in range(0,len(df2['soft_consine_score'])):
  if 'pos' in df2['soft_consine_score'][i]:
    df2['predict_sentiment'][i]='positive'
  if 'neg' in df2['soft_consine_score'][i]:
    df2['predict_sentiment'][i]='negative'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(df2.category +', ' + df2.final_sentiment +', ' + df2.predict_category + ', ' + df2.predict_sentiment)

0     4, positive, 1, positive
1     4, positive, 1, positive
2      4, unknown, 4, positive
3     4, positive, 4, positive
4     4, positive, 4, positive
5     4, positive, 1, positive
6     4, positive, 1, positive
7     4, positive, 4, positive
8     4, negative, 1, negative
9     4, positive, 4, positive
10     3, unknown, 3, positive
11    3, positive, 3, positive
12     3, unknown, 4, negative
13    3, positive, 4, negative
14    3, positive, 2, negative
15    3, positive, 2, positive
16    3, positive, 1, positive
17    3, positive, 4, positive
18    3, positive, 4, negative
19    3, positive, 1, positive
20    3, positive, 4, negative
21    3, positive, 2, positive
22    3, positive, 2, positive
23    3, positive, 3, positive
24    3, positive, 2, positive
25    3, negative, 3, positive
26    3, positive, 1, positive
27    3, positive, 3, positive
28    3, positive, 2, positive
29    3, positive, 3, positive
30    3, negative, 3, negative
31    3, positive, 4, negative
32    2,

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(df2['category'], df2['predict_category']))
print(accuracy_score(df2['final_sentiment'], df2['predict_sentiment']))

0.3
0.64


In [ ]:
#https://www.machinelearningplus.com/nlp/cosine-similarity/